In [1]:
import pandas as pd
from dmyplant2 import cred, MyPlant, Engine, msgFSM, plot_with_additional_results, FSM, bokeh_show
from pprint import pprint as pp
    
cred()
mp = MyPlant(0)
Engine._list_cached_validations();

In [2]:
e=Engine.from_sn(mp,1486174)
modes = ['???','OFF','MANUAL','AUTO']
success = [True,False]
e['IB Site Name'] + ' ' + e['Engine ID'], modes, success

('Forsa Hartmoor M04', ['???', 'OFF', 'MANUAL', 'AUTO'], [True, False])

In [3]:
fsm = msgFSM(e, p_from=e['Commissioning Date'], p_to='2022-01-15', successtime=300)
fsm.run1(enforce=True, silent=True) # run Finite State Machine

In [4]:
rda = fsm.starts
rda['stime'] = rda.apply(lambda x: x['starttime'].strftime('%d.%m.%Y %H:%M:%S'), axis=1)
rda['etime'] = rda.apply(lambda x: x['endtime'].strftime('%d.%m.%Y %H:%M:%S'), axis=1)
rda[['no','success','mode','stime','etime','startpreparation','starter','speedup','idle','synchronize','loadramp','cumstarttime','targetoperation','rampdown','coolrun','runout']].round(2)

,no,success,mode,stime,etime,startpreparation,starter,speedup,idle,synchronize,loadramp,cumstarttime,targetoperation,rampdown,coolrun,runout
0,0,True,???,11.01.2022 09:54:21,11.01.2022 14:57:18,97.95,4.24,24.00,6.35,13.02,126.89,272.46,17718.31,127.73,9.99,48.58
1,1,True,???,11.01.2022 15:33:10,11.01.2022 15:50:37,92.73,3.84,20.03,5.56,28.36,126.82,277.35,584.11,127.33,9.99,48.82
2,2,False,MANUAL,11.01.2022 18:37:19,11.01.2022 18:44:41,442.40,NaN,NaN,NaN,NaN,NaN,442.40,NaN,NaN,NaN,NaN
3,3,False,AUTO,11.01.2022 18:56:43,11.01.2022 19:08:22,76.68,3.84,21.80,5.54,24.14,126.83,258.82,254.80,127.14,9.99,48.65
4,4,False,AUTO,12.01.2022 11:22:41,12.01.2022 11:24:53,106.12,4.14,21.08,NaN,NaN,NaN,131.34,NaN,NaN,NaN,NaN
5,5,True,AUTO,13.01.2022 02:51:02,13.01.2022 04:03:09,96.41,4.03,23.10,6.25,9.58,126.44,265.83,3875.23,127.46,10.01,48.66
6,6,True,AUTO,13.01.2022 04:11:15,13.01.2022 08:49:43,60.61,3.63,18.89,5.15,28.75,126.51,243.53,16279.04,127.52,9.99,48.25
7,7,True,AUTO,14.01.2022 09:58:05,14.01.2022 10:33:09,96.53,3.94,24.82,6.16,8.79,126.60,266.84,1651.12,127.31,9.90,48.35
8,8,True,AUTO,14.01.2022 16:28:05,14.01.2022 19:33:11,99.23,3.93,22.48,6.07,9.49,126.51,267.71,10652.03,127.53,10.00,48.94


In [5]:
vset = ['Various_Values_SpeedAct','Power_PowerAct','Hyd_PressCrankCase','Hyd_PressOilDif','Hyd_PressOil','Hyd_TempOil']
dset = [{'col':['Power_PowerAct'], 'ylim':(0,5000), 'color':'red'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue'},
        {'col':['Hyd_PressCrankCase'],'ylim': [-100, 100], 'color':'orange'},
        {'col':['Hyd_PressOilDif'],'ylim': [0, 2], 'color':'black'},
        {'col':['Hyd_PressOil'],'ylim': [0, 10], 'color':'brown'},
        {'col':['Hyd_TempOil'],'ylim': [0, 110], 'color':'#2171b5'}]
startversuch = rda.iloc[6]; 
fig = plot_with_additional_results(fsm,startversuch, vset, dset, dfigsize=(16,10))
bokeh_show(fig)


von: 13.01.2022 04:11:15 bis: 13.01.2022 08:49:43


In [6]:
startversuch=fsm.starts.iloc[6].to_dict()
#pp(startversuch)

In [7]:
for i, v in enumerate(fsm.runlogdetail(startversuch, statechanges_only=False)):
    print(f"{i:3} {v}")

  0 * |0006| 13.01 04:03:09 standstill        | 13.01 04:11:15 startpreparation  | off | AUTO  | 600 13.01.2022 04:11:15 1231 Request module on
  1   |0007| 13.01 04:03:09 standstill        | 13.01 04:11:15 startpreparation  | on  | AUTO  | 600 13.01.2022 04:12:05 9007 Hourly message
  2 * |0007| 13.01 04:11:15 startpreparation  | 13.01 04:12:16 starter           | on  | AUTO  | 600 13.01.2022 04:12:16 1249 Starter on
  3   |0007| 13.01 04:11:15 startpreparation  | 13.01 04:12:16 starter           | on  | AUTO  | 600 13.01.2022 04:12:18 1277 Demand exhaust gas scavenging fan on
  4   |0007| 13.01 04:11:15 startpreparation  | 13.01 04:12:16 starter           | on  | AUTO  | 600 13.01.2022 04:12:18 1279 Exhaust gas scavenging fan operation on
  5 * |0007| 13.01 04:12:16 starter           | 13.01 04:12:19 speedup           | on  | AUTO  | 600 13.01.2022 04:12:19 3225 Ignition on
  6   |0007| 13.01 04:12:16 starter           | 13.01 04:12:19 speedup           | on  | AUTO  | 600 13.01.2022

In [ ]:
print(fsm.results['runlogdetail'][0])
print(fsm.results['runlogdetail'][-1])

In [ ]:
for sv in fsm.results['runlogdetail']:
    print(sv)

In [ ]:
print(len(fsm.results['runlogdetail']))